Checking the goal using the distance and numpy

##Download the model from Roboflow

In [ ]:
##MODEL TO DETECT GOALPOSTS
from inference import get_model

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

GOALPOST_DETECTION_MODEL_ID = "football-goalpost/3"
GOALPOST_DETECTION_MODEL = get_model(model_id=GOALPOST_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

In [ ]:
##MODEL TO DETECT PLAYERS AND BALL AND REFREES
from inference import get_model

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

PLAYER_DETECTION_MODEL_ID = "football-players-detection-3zvbc/12"
PLAYER_DETECTION_MODEL = get_model(model_id=PLAYER_DETECTION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

In [ ]:
##MODEL TO CLASSIFY GOAL OR NO GOAL MOMMENT 
from inference import get_model

ROBOFLOW_API_KEY = "ITfUpuY5QO9WTBpcEXTh"  # Replace with your actual Roboflow API key

CLASSIFICATION_MODEL_ID = "goal-ckeck-classification/4"
CLASSIFICATION__MODEL = get_model(model_id=CLASSIFICATION_MODEL_ID, api_key=ROBOFLOW_API_KEY)

##import the dependecies

In [ ]:
import cv2
import numpy as np  
import supervision as sv

In [ ]:
image_path = "C:/Users/user/Desktop/FootballAI/videos/ronaldo1.png"  # Path to the image


##DETECT THE GOALPOST SPERATELY


In [ ]:

goalpost_center = None  # Variable to store the center of the goalpost

image = cv2.imread(image_path)  # Load the image using OpenCV

result = GOALPOST_DETECTION_MODEL.infer(image, confidence=0.3)[0]

detections = sv.Detections.from_inference(result)
print(detections.xyxy)

for detection, class_id in zip(detections.xyxy, detections.class_id):
        x_min, y_min, x_max, y_max = detection
        goalpost_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Calculate the center of the goalpost
        print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates
        # Draw a red dotted circle at the center
        center_coordinates = (int(goalpost_center[0]), int(goalpost_center[1]))
        radius = 5
        color = (0, 0, 255)  # Red color in BGR
        thickness = 2
        line_type = cv2.LINE_AA  # Anti-aliased line for better quality
        cv2.circle(image, center_coordinates, radius, color, thickness, lineType=line_type)
        
        break  # Stop after finding the first goalpost

# Display the image with the red dotted circle
cv2.imshow("Image with Center Point", image)
cv2.waitKey(0)
cv2.destroyAllWindows()



##DETECT THE BALL SPERATELY


In [ ]:

ball_center = None  # Variable to store the center of the ball
image = cv2.imread(image_path)  # Load the image using OpenCV
BALL_ID = 0
result = PLAYER_DETECTION_MODEL.infer(image, confidence=0.2)[0]

detections = sv.Detections.from_inference(result)

for detection, class_id in zip(detections.xyxy, detections.class_id):
    if class_id == BALL_ID:
        x_min, y_min, x_max, y_max = detection
        ball_center = ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Calculate the center of the ball
        print(f"Ball center: {ball_center}")  # Print the center coordinates

# Check if ball_center is not None before using it
if ball_center is not None:
    center_coordinates = (int(ball_center[0]), int(ball_center[1]))
    radius = 5
    thickness = 2
    line_type = cv2.LINE_AA  # Anti-aliased line for better quality
    cv2.circle(image, center_coordinates, radius, (0, 0, 255), thickness, line_type)  # Draw the circle

cv2.imshow("Image with Center Point", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Draw a red dotted circle at the center of the goalpost

In [ ]:
if goalpost_center is not None:
    goalpost_center_coordinates = (int(goalpost_center[0]), int(goalpost_center[1]))
    cv2.circle(image, goalpost_center_coordinates, radius, (0, 0, 255), thickness, lineType=line_type)

# Draw a red dotted circle at the center of the ball
if ball_center is not None:
    ball_center_coordinates = (int(ball_center[0]), int(ball_center[1]))
    cv2.circle(image, ball_center_coordinates, radius, (0, 0, 255), thickness, lineType=line_type)

# Display the image with both circles
cv2.imshow("Image with Goalpost and Ball Centers", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates of the goalpost
print(f"Ball center: {ball_center}")  # Print the center coordinates of the ball

##Calculate the Euclidean distance between goalpost_center and ball_center


In [ ]:
distance = np.sqrt((goalpost_center[0] - ball_center[0])**2 + (goalpost_center[1] - ball_center[1])**2)

# Define a threshold for "close enough" (you can adjust this value as needed)
threshold =50  # Example threshold

# Determine if it's a goal
if distance <= threshold:
    result = "Goal"
else:
    result = "Not Goal"

print(f"Distance: {distance}")
print(f"Result: {result}")

    ## Classify the Goal with classification model

In [ ]:

    
def classify_goal():
        classification_result = CLASSIFICATION__MODEL.infer(image)  # Pass the image to the classifier
        goal_prediction = classification_result[0]  # Access the first prediction from the list
        goal_label = goal_prediction.predicted_classes[0]  # Get the predicted class
        goal_probability = goal_prediction.predictions[goal_label].confidence  # Get the confidence score
        return goal_label,goal_probability

goal_label, goal_probability = classify_goal()  # Use the correct attribute for the confidence score
    
    print(f"Classification Model Prediction: {goal_label} (Confidence: {goal_probability * 100:.2f}%)")
    
    if goal_label == "goal":
        print("Goal detected!")
    elif goal_label == "nogoal":
        print("No Goal detected.")
    else:
        print("Ball or goalpost not detected.")

##generating video detect the goal by calculating the distance between the goalpost and the ball , and if the distance close enough it classify the frame and double check the goal frame

In [ ]:
# Path to the video file
video_path = "C:/Users/user/Desktop/FootballAI/videos/english.mp4"

# Path to save the resulting video
output_video_path = "C:/Users/user/Desktop/FootballAI/videos/resulting_video5.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Initialize the video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4 format
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Initialize the goal counter
goal_counter = 0

# Loop over frames in the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit the loop if no more frames are available
   # goalpost_center = infer_goalpost_center(frame)  # Call the function to get the goalpost center
   # ball_center = infer_ball_center(frame)  # Call the function to get the ball center

    distance = np.sqrt((goalpost_center[0] - ball_center[0])**2 + (goalpost_center[1] - ball_center[1])**2)  # Calculate the distance

    threshold = 50  # Example threshold for "close enough"

    if distance <= threshold:
       # Use the classification model to double-check
        classification_result = CLASSIFICATION__MODEL.infer(frame)  # Pass the image to the classifier
        goal_prediction = classification_result[0]  # Access the first prediction from the list
        goal_label = goal_prediction.predicted_classes[0]  # Get the predicted class
        goal_probability = goal_prediction.predictions[goal_label].confidence  # Get the confidence score

        if goal_label == "goal" and goal_probability > 0.8 and goal_counter == 0: #Confidence threshold for classification
            goal_counter += 1 # Increment the goal counter

    # Display the frame with the goal counter
    cv2.putText(frame, f"Goals Scored: {goal_counter}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    # Write the frame to the output video
    out.write(frame)

    # Optionally display the frame (can be commented out if not needed)
    cv2.imshow("Video", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer objects, and close all OpenCV windows
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Resulting video saved at: {output_video_path}")

##methods to get the centers of goalpost and ball

In [ ]:
 video_path = "C:/Users/user/Desktop/FootballAI/videos/english.mp4"
 frame_generator = sv.get_video_frames_generator(video_path)
 frame = frame_generator.__next__() 
 BALL_ID=0

def infer_goalpost_center(frame):
    
    goalpost_result = GOALPOST_DETECTION_MODEL.infer(frame, confidence=0.3)[0]
    goalpost_detections = sv.Detections.from_inference(goalpost_result)
    for detection in goalpost_detections.xyxy:
        x_min, y_min, x_max, y_max = detection
        return ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Return the center of the goalpost
    return None  # Return None if no goalpost is found

# Use the function to infer the goalpost center
goalpost_center = infer_goalpost_center(frame)

def infer_ball_center(frame):
    ball_result = PLAYER_DETECTION_MODEL.infer(frame, confidence=0.5)[0]
    ball_detections = sv.Detections.from_inference(ball_result)
    for detection, class_id in zip(ball_detections.xyxy, ball_detections.class_id):
        if class_id == BALL_ID:
            x_min, y_min, x_max, y_max = detection
            return ((x_min + x_max) / 2, (y_min + y_max) / 2)  # Return the center of the ball
    return None  # Return None if no ball is found

# Use the function to infer the ball center
ball_center = infer_ball_center(frame)
print(f"Goalpost center: {goalpost_center}")  # Print the center coordinates of the goalpost
print(f"Ball center: {ball_center}")  # Print the center coordinates of the ball